In [21]:
import mongo_queries
from bson.objectid import ObjectId

In [25]:
sites = [427,440,602,351,453,840]

In [26]:
import datetime as dt

In [27]:
today = dt.datetime.today()

In [28]:
curr = mongo_queries.current

In [29]:
doc = {'datetime' : today, 'sites':sites}

In [30]:
curr.insert_one(doc)

In [31]:
list(curr.find({}).sort('datetime',-1).limit(1))[0]['sites']

[427, 440, 602, 351, 453, 840]

In [32]:
mongo_queries.get_updating_list()

[427, 440, 602, 351, 453, 840]

In [33]:
base = mongo_queries.col

In [53]:
late = mongo_queries.late
leg = mongo_queries.leg
run = mongo_queries.run

In [35]:
id = 817

In [42]:
one = base.find_one({'id':id})

In [52]:
uuid = one['_id']

In [76]:
def migrate_times_by_id(id):
    waits = list(run.find({'crossing_id':id},projection={'utc':1,'wait':1, '_id':0}))
    
    new_vals = {'$set':{'wait_times':waits}}
    upd =  base.update_one({'id':id},new_vals)
    print(upd)
    return upd
    

In [77]:
mig = migrate_times_by_id(id)

In [79]:
mig.acknowledged

True

In [80]:
mig.matched_count

1

In [81]:
mig.modified_count

1

In [82]:
mig.raw_result

{'n': 1,
 'nModified': 1,
 'opTime': {'ts': Timestamp(1637253792, 31), 't': 218},
 'electionId': ObjectId('7fffffff00000000000000da'),
 'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1637253792, 31),
  'signature': {'hash': b'2\x8b\xcf\xde\xdb\x1d\xa3[\x12\x1f\x1e=\xca\xa5\xbe\x88`e\xc2\x82',
   'keyId': 6995538273825193985}},
 'operationTime': Timestamp(1637253792, 31),
 'updatedExisting': True}

In [67]:
wt = base.find_one({'id':id})['wait_times']

In [68]:
import pandas as pd

In [70]:
pd.DataFrame(wt)

,utc,wait
0,2021-08-30 20:20:00.000,93
1,2021-08-30 20:30:00.000,0
2,2021-08-30 20:40:00.000,0
3,2021-08-30 20:50:00.000,4
4,2021-08-30 21:00:00.000,10
...,...,...
787,2021-10-04 15:34:32.342,0
788,2021-10-04 16:05:51.313,0
789,2021-10-04 16:36:53.644,0
790,2021-10-04 17:07:53.837,0


In [122]:
ids = base.find({'wait_times':{'$exists':False}},projection={'id':1,'_id':0})

In [123]:
my_list = [i['id'] for i in list(ids)]

In [124]:
my_list

[]

In [125]:
import pandas as pd

In [15]:
def make_filter_for_baseline_by_id_list(ids,filt=None):
    mini = None

    if type(ids) == list:
        if len(ids) == 1:
            mini = {'id':ids[0]}
        elif len(ids) > 1:
            mini = {'id':{'$in':ids}}
    else:
        mini = {'id':ids}
    if filt:
        print('filter found',filt, type(filt))
        print('mini:', mini, type(mini))
        res = filt.update(mini)
        print(res)
        return res
    else:
        return mini

In [17]:
filt = {'Region':'Pacific'}

In [18]:
make_filter_for_baseline_by_id_list(1234, filt)

filter found {'Region': 'Pacific'} <class 'dict'>
mini: {'id': 1234} <class 'dict'>
None


In [23]:
items = mongo_queries.get_updating_list()

In [2]:
# db = cluster['bordercross']
# col= db['baseline']
# run = db['running']
# leg = db['legacy']
# late = db['latest times']
# #merge = db['running_merge']
# merge = db['running_merge2'] #for smaller queries
# lm = db['latest_merged']
# current = db['current_updaters']

col = mongo_queries.col
run = mongo_queries.run
leg = mongo_queries.leg
current = mongo_queries.current

In [55]:
import pytz

In [68]:
def get_last_run_base(id, req = None):
    ###Given an id, return the last reading
    ###Part of DB reset, times are now sub doc arrays on the baseline collection
    res = col.aggregate([
    {
        '$match': {
            'id': id
        }
    }, {
        '$unwind': {
            'path': '$wait_times', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$project': {
            '_id': 0, 
            'id': 1, 
            'name': 1, 
            'province': 1, 
            'dest': 1, 
            'timeZone': 1, 
            'region': 1, 
            'district': 1, 
            'utc': '$wait_times.utc', 
            'wait': '$wait_times.wait'
        }
    }, {
        '$sort': {
            'utc': -1
        }
    }, {
        '$limit': 1
    }
])
    res = list(res)[0]
    if req == 'wait':
        return res['wait']
    elif req =='utc':
        return res['utc'].astimezone(pytz.utc)
    elif req =='local':
        utc = res['utc']
        local_tz = res['timeZone']
        
        return utc.astimezone(pytz.timezone(local_tz))
    else:
        return res

In [69]:
a = get_last_run_base(840,'local')
a

datetime.datetime(2021, 11, 18, 14, 10, tzinfo=<DstTzInfo 'America/Vancouver' PST-1 day, 16:00:00 STD>)

In [70]:
a

datetime.datetime(2021, 11, 18, 14, 10, tzinfo=<DstTzInfo 'America/Vancouver' PST-1 day, 16:00:00 STD>)

In [71]:
a = get_last_run_base(840,'utc')
a

datetime.datetime(2021, 11, 18, 22, 10, tzinfo=<UTC>)

In [1]:
import mongo_queries

In [4]:
l = mongo_queries.get_updating_list()

In [5]:
mongo_queries.get_last_run_base(l[0])

{'id': 427,
 'name': 'Queenston Lewiston Bridge',
 'province': 'ON',
 'dest': '43.154644, -79.049258',
 'timeZone': 'America/Toronto',
 'region': 'Southern Ontario',
 'district': 'Niagara Falls',
 'utc': datetime.datetime(2021, 11, 18, 17, 10),
 'wait': 102}